In [15]:
from py2neo import Graph, Path
from py2neo import Node, Relationship
from pymongo import MongoClient
import json

config = json.load(open("config.json"))

graph = Graph("http://"+config["user"]+":"+config["password"]+"@"+config["server"]+":7474")

client = MongoClient()
client = MongoClient(config["server"], 27017)
db = client.KG

## First add categories in the graph

In [2]:
def clean(somestring):
    return somestring.replace("'","/")

In [44]:
def transintoneo(edges):
    for edge in edges:
        if edge['content'] != "added via":
            pname = 'Source'
            cname = 'Link'
            rname = clean(edge['content'])
            contype = 'category'
            cat = "Pages"
            transition(pname, cname, rname, contype, cat, cat, edge)
        else:
            pname = 'Parent'
            cname = 'Child'
            rname = clean(edge['content'])
            contype = 'type'
            cat = "Category"
            transition(pname, cname, rname, contype, cat, cat, edge)
        
def transition(pname, cname, rname, contype, catP, catC, edge, Child):
    #if edge[cname] is not None:
    if True:
        Parent = clean(edge[pname]['label'][0])
        #Child = clean(edge[cname]['title'])
        Child = clean(Child)
        queryP = "MATCH (c:"+catP+") WHERE c.title='" + Parent + "' RETURN c"
        queryC = "MATCH (c:"+catC+") WHERE c.title='" + Child + "' RETURN c"
        if len(graph.run(queryP).data()) == 0:
            Pa = Node(catP,title = Parent,content = edge[pname][contype])
            graph.create(Pa)
            #if cat == "Pages":
                #addpagecategory(edge[pname])
        else:
            Pa = graph.run(queryP).data()
        if len(graph.run(queryC).data()) == 0:
            Ch = Node(catC,title = Child,content = edge[cname][contype])
            graph.create(Ch)
            #if cat == "Pages":
                #addpagecategory(edge[cname])
        else:
            Ch = graph.run(queryC).data()
        queryR = "MATCH (p:"+catP+")-[r:`"+rname+"`]->(c:Category) WHERE (p.title='"+Parent+\
                    "' AND c.title='"+Child+"') RETURN p"
        if len(graph.run(queryR).data()) == 0:
            createR = "MATCH (p:"+catP+" {title:'"+Parent+"'}),(c:"+catC+" {title:'"+Child+"'})" \
            "CREATE (c)-[:`"+rname+"`]->(p)"
            #print(createR)
            graph.run(createR)
        else:
            pass
    
def addpagecategory(page):
    for cat in page['category']:
        createR = "MATCH (p:Pages {title:'"+clean(page['title'])+"'}),(c:Category {title:'"+clean(cat)+"'})" \
            "CREATE (c)-[:`Belongs To`]->(p)"
        graph.run(createR)
        

In [37]:
categoriesEdges = db.cat_edges.find({})
transintoneo(categoriesEdges)

In [14]:
pagesEdges = db.page_edges.find({}).limit(2000)
transintoneo(pagesEdges)

## Adding the data with dbpedia label into neo4j

In [43]:
def addedge(edge):
    pname = 'Source'
    cname = 'Destination'
    rname = clean(edge['relationship'])
    contype = 'label'
    catP = edge['Source']['category']
    catC = edge['Destination']['category']
    for Child in edge['Destination']['label']:
        transition(pname, cname, rname, contype, catP, catC, edge, Child)

In [45]:
def visualization():
    edges = db.edges.find({})
    totalamount = db.edges.find({}).count()
    count = 0
    for edge in edges:
        addedge(edge)
        if count % 100 == 0:
            print("The process is ", count)
        count += 1

In [47]:
visualization()

The process is  0
The process is  100
The process is  200
The process is  300
The process is  400
The process is  500
The process is  600
The process is  700
The process is  800
The process is  900
The process is  1000
The process is  1100
